# AG3 - Gonzalo Pérez Vizuete
### Github: https://github.com/comandantexd/03MAIR_Algoritmos_optimizacion/blob/master/AG3.ipynb

# Precarga, imports y variables

In [3]:
import urllib
import tsplib95
import random
import math
import copy
import numpy as np

file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file) 

#Objeto problem
problem = tsplib95.load(file)

#Nodos
nodes = list(problem.get_nodes())

#Aristas
edges = list(problem.get_edges())

# Utilidades

In [88]:
#Genera una colucion aleatoria con comienzo en el nodo 0
def generate_solution(nodes): 
    sol = [nodes[0]]
    for n in nodes[1:]:
        sol = sol + [random.choice(list(set(nodes) - set({nodes[0]}) - set(sol)))]

    return sol

#Distancia entre dos nodos
def distance(a,b, problem):
    return problem.get_weight(a, b)

#Distancia total de una trayectoria/solucion
def total_distance(sol, problem):
    dist = 0
    for i in range(len(sol) - 1):
        dist += distance(sol[i], sol[i + 1], problem)

    return dist + distance(sol[len(sol) - 1], sol[0], problem)

# Búsqueda aleatoria

In [89]:
def random_search(problem, n):
    nodes = list(problem.get_nodes())

    best_sol = []
    best_dist = 10e100

    for i in range(n):
        sol = generate_solution(nodes)
        dist = total_distance(sol, problem)

        if dist < best_dist:
            best_sol = sol
            best_dist = dist
    
    return best_sol, best_dist

sol, dist = random_search(problem, 50000)    

print('Mejor solución: {}'.format(sol))
print('Distancia : {}'.format(dist))
 

Mejor solución: [0, 11, 41, 23, 29, 10, 4, 13, 6, 17, 37, 25, 40, 21, 24, 30, 38, 28, 15, 12, 7, 31, 32, 20, 5, 18, 34, 16, 19, 14, 1, 2, 22, 3, 33, 36, 35, 26, 39, 9, 8, 27]
Distancia : 3471


In [90]:
def generate_neighbor(solution, problem):
    #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones

    best_sol = []
    best_dist = 10e100

    for i in range(1, len(sol) - 1): #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
        for j in range(i + 1, len(sol)): #i+1 por la simetríA
      
            # Generar una nueva solucion intercambaindo nodos
            neighbor = solution[:i] + [solution[j]] + solution[i+1:j] + [solution[i]] + solution[j+1:]

            n_dist = total_distance(neighbor, problem) #Evaluar la solucion

            if n_dist <= best_dist:
                best_dist = n_dist
                best_sol = neighbor

    return best_sol



new_sol = generate_neighbor(sol, problem)

print('Distancia antigua Solucion: {}'.format(total_distance(sol, problem)))
print('Distancia nueva Solucion: {}'.format(total_distance(new_sol, problem)))

Distancia antigua Solucion: 3471
Distancia nueva Solucion: 3195


# Búsqueda local

In [91]:
def local_search(problem, nodes):
    best_sol = []
    #Generar una solucion inicial de referencia(aleatoria)
    sol_ref = generate_solution(nodes)
    best_dist = total_distance(sol_ref, problem)

    finish = False
    iter = 0
    while not finish:
        iter += 1
        neighbor = generate_neighbor(sol_ref, problem)
        n_dist = total_distance(neighbor, problem)

        if n_dist < best_dist:
            best_sol = copy.deepcopy(neighbor)
            best_dist = n_dist
        else:
            print('Iteracion: %d' % iter)
            finish = True #Terminamos, significa que es un mínimo local
        
        sol_ref = neighbor

    return best_sol, best_dist

sol, dist = local_search(problem, nodes)
print('Mejor solución: {}'.format(sol))
print('Distancia total: {}'.format(dist))

Iteracion: 47
Mejor solución: [0, 20, 33, 34, 38, 22, 23, 41, 10, 25, 11, 12, 13, 19, 14, 16, 15, 6, 4, 2, 27, 3, 1, 7, 17, 31, 28, 30, 32, 35, 36, 37, 5, 26, 18, 8, 9, 21, 40, 24, 39, 29]
Distancia total: 1812


In [93]:
#Generador de 1 solucion vecina 2-opt aleatoria (intercambiar 2 nodos)
def generate_random_neighbor(solution):
    #Se eligen dos nodos aleatoriamente
    i, j = sorted(random.sample( range(1, len(solution)), 2))

    #Swap de nodos
    return solution[:i] + [solution[j]] + solution[i+1:j] + [solution[i]] + solution[j+1:]
  
 
print(generate_random_neighbor(sol) )

[0, 20, 33, 34, 38, 22, 23, 41, 10, 25, 11, 12, 13, 19, 14, 16, 3, 6, 4, 2, 27, 15, 1, 7, 17, 31, 28, 30, 32, 35, 36, 37, 5, 26, 18, 8, 9, 21, 40, 24, 39, 29]


# Recocido simulado

In [94]:
#Funcion de probabilidad para aceptar peores soluciones
def probability(T, d):
    return random.random() < math.exp(-1 * d / T)


#Funcion de descenso de temperatura
def lower_temp(T):
    return T * 0.99

    
def simulated_annealing(problem, temp, nodes):
    sol_ref = generate_solution(nodes)
    dist_ref = total_distance(sol_ref, problem)
  
    best_sol = []
    best_dist = 10e100
  
  
    n = 0
    while temp > 0.0001:
        n += 1
        #Genera una solución vecina
        neighbor = generate_random_neighbor(sol_ref)
    
        #Calcula su valor(distancia)
        n_dist = total_distance(neighbor, problem)
      
        #Si es la mejor solución de todas se guarda(siempre!!!)
        if n_dist < best_dist:
            best_sol = neighbor
            best_dist = n_dist
    
        #Si la nueva vecina es mejor se cambia  
        #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
        if n_dist < dist_ref or probability(temp, abs(dist_ref - n_dist)):
            sol_ref = copy.deepcopy(neighbor)
            dist_ref = n_dist

        #Baja temperatura
        temp = lower_temp(temp)


    return best_sol, best_dist

sol, dist  = simulated_annealing(problem, 100000000000, nodes)
 
print('Mejor solución: {}'.format(sol))
print('Distancia total: {}'.format(dist))

Mejor solución: [0, 5, 26, 6, 41, 40, 24, 21, 9, 23, 29, 28, 1, 16, 15, 14, 19, 13, 7, 20, 33, 34, 32, 27, 2, 3, 17, 31, 35, 36, 37, 18, 10, 8, 25, 11, 12, 4, 30, 39, 22, 38]
Distancia total: 2073


# Colonia de hormigas

In [210]:
l = len(nodes)

def select_node(pheromones, current_road):
    not_index = [v for v in current_road if v != -1]

    # Influencia de cada valor (alfa: feromonas; beta: valor)
    alfa = 1.0
    beta = 0.5

    _p = []
    for i, v in enumerate(pheromones):
        p = math.pow((1.0 + v), alfa)
        w = math.pow(1.0 / p, beta) * p
        _p.append(p if i not in not_index else 0)

    acum = 0
    value = random.random() * sum(_p)
    i = -1
    while value > acum:
        i += 1
        acum += _p[i]
    
    return i


def select_road(problem, pheromones):
    road = [-1]*l
    for i in range(l):
        road[i] = select_node(pheromones[i], road)

    return road


def update_pheromones(pheromones, problem, road, p):
    for i, p_list in enumerate(pheromones):
        n = road[i]
        p_list[n] += p # Los nodos mas lejos obtienen menos feromonas


def evaporate_pheromones(pheromones):
    for p_list in pheromones:
        for j, v in enumerate(p_list):
            p_list[j] *= 0.9 # Evapora las feromonas


def ACO_metaheuristic(problem, nodes, iter):
    pheromones = []
    for i in range(l):
        pheromones.append([0]*l) #initialize pheromones empty array

    max_dist = max(max(problem.edge_weights))
    n_nodes = len(nodes)
    mean_dist = n_nodes * max_dist / 2


    t_dist = 10e100
    best_road = []

    i = 0
    while i < iter: #iteraciones
        road = select_road(problem, pheromones)
        dist = total_distance(road, problem)

        if dist < t_dist:
            t_dist = dist
            best_road = road
            update_pheromones(pheromones, problem, road, mean_dist / t_dist)
        
        evaporate_pheromones(pheromones)

        i += 1

    return t_dist, best_road

dist, sol = ACO_metaheuristic(problem, nodes, 100)
print('iteraciones: 100')
print('Mejor solución: {}'.format(sol))
print('Distancia total: {}'.format(dist))


dist, sol = ACO_metaheuristic(problem, nodes, 10000)
print('\niteraciones: 10000')
print('Mejor solución: {}'.format(sol))
print('Distancia total: {}'.format(dist))

iteraciones: 100
Mejor solución: [14, 34, 38, 2, 21, 41, 10, 29, 32, 13, 18, 9, 28, 33, 19, 36, 17, 24, 11, 37, 12, 22, 6, 7, 15, 35, 25, 8, 39, 40, 4, 0, 16, 31, 1, 30, 23, 27, 20, 3, 26, 5]
Distancia total: 4212

iteraciones: 10000
Mejor solución: [39, 22, 30, 27, 12, 23, 5, 6, 16, 15, 14, 37, 34, 26, 25, 11, 41, 24, 33, 1, 21, 20, 32, 19, 7, 31, 4, 29, 2, 10, 13, 17, 18, 40, 38, 28, 3, 36, 35, 0, 8, 9]
Distancia total: 3595
